Updating the SED files for MUSCLES to newer MAST standards - modified from the same notebook for MEATS

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
import os
import glob
from astropy.table import Table
from astropy.io import ascii
from astropy.convolution import convolve, Box1DKernel
from scipy.interpolate import interp1d

import astropy.units as u
from astropy import time, coordinates as coord
import astropy.constants as const
from astropy.modeling import models, fitting
from astropy.coordinates import SkyCoord
from matplotlib.gridspec import GridSpec
from datetime import datetime
from astroquery.simbad import Simbad

#matplotlib set up
%matplotlib inline
from matplotlib import rcParams
rcParams["figure.figsize"] = (14, 5)
rcParams["font.size"] = 20

In [39]:
oldv = 24
version = 25 #v25 is now the MUSCLES update, MEATS will have to be v26


    STIS spectra

DOI = ‘10.17909/T9DG6F’

EQUINOX =’2000.2’ or RADESYS = 'ICRS'

HLSPID =’MUSCLES’

HLSPVER=’V1’

LICENSE =’CC BY 4.0’

LICENSURL= 'https://creativecommons.org/licenses/by/4.0/'

TIMESYS: Character string code (see FITS Standard) for the time scale of time-related keywords. Ex: 'UTC'

 

    Model spectra

The same set of metadata is required as STIS data except for EQUINOX/RADESYS and TIMESYS.

 

    SED data 

The same set of metadata is required as STIS data except for OBSERVAT.

MJD-BEG and MJD-END in the primary header. (* see below)

 

    XMM data

The same set of metadata is required as STIS data.

RA_TARG and DEC_TARG values are missing. Please include the correct RA/Dec values.

In [35]:
inpath = '/media/david/2tb_ext_hd/hddata/mega_muscles/v24_hlsp/'
outpath = '/media/david/2tb_ext_hd/hddata/mega_muscles/v26_hlsp_fixed/'
stars = os.listdir(inpath)
# print(stars)
stars = np.array(stars)
# mask = stars != '.ipynb_checkpoints'
# print(mask)
# stars = stars[mask]
star_fix = np.copy(stars)  #change to simbad recognised names
for i in range(len(stars)):
    if stars[i] == 'L-980-5':
        star_fix[i] = 'L 980-5'
    if stars[i] == 'LHS-2686':
        star_fix[i] = 'LHS 2686'
    
print(stars)
print(len(stars))

['GJ1132' 'GJ15A' 'GJ163' 'GJ649' 'GJ674' 'GJ676A' 'GJ699' 'GJ729' 'GJ849'
 'L-980-5' 'LHS-2686' 'TRAPPIST-1']
12


In [12]:
star_list = Simbad.query_objects(star_fix)
star_list

MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE,SCRIPT_NUMBER_ID
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,,
object,str13,str13,int16,int16,float32,float32,int16,str1,str1,object,int32
L 320-124,10 14 51.7786,-47 09 24.192,14,14,0.014,0.015,90,A,O,2020yCat.1350....0G,1
HD 1326,00 18 22.8849,+44 01 22.637,14,14,0.014,0.010,90,A,O,2020yCat.1350....0G,2
L 229-91,04 09 15.6683,-53 22 25.290,14,14,0.016,0.017,90,A,O,2020yCat.1350....0G,3
BD+25 3173,16 58 08.8496,+25 44 38.974,14,14,0.016,0.019,90,A,O,2020yCat.1350....0G,4
CD-46 11540,17 28 39.9455,-46 53 42.688,14,14,0.025,0.018,90,A,O,2020yCat.1350....0G,5
CD-51 10924,17 30 11.2044,-51 38 13.130,14,14,0.027,0.023,90,A,O,2020yCat.1350....0G,6
NAME Barnard's star,17 57 48.4984,+04 41 36.113,14,14,0.026,0.029,90,A,O,2020yCat.1350....0G,7
CD-23 14742,18 49 49.3637,-23 50 10.447,14,14,0.029,0.026,90,A,O,2020yCat.1350....0G,8


In [15]:

stars[9:]

array(['L-980-5', 'LHS-2686', 'TRAPPIST-1'], dtype='<U10')

In [28]:

newpath = outpath
for star in stars[11:]:
    print(star)
    newdir = '{}{}'.format(newpath, star)
    if not os.path.exists(newdir):
       os.makedirs(newdir)
    
    oldv = 24

    starpath = '{}{}/'.format(inpath, star)
    specs = np.hstack((glob.glob('{}*stis*.fits'.format(starpath)),glob.glob('{}*cos*.fits'.format(starpath))))
    mods = glob.glob('{}*mod*.fits'.format(starpath))
    seds = glob.glob('{}*multi*.fits'.format(starpath))
    xrays = np.hstack((glob.glob('{}*cxo*.fits'.format(starpath)),glob.glob('{}*xmm*.fits'.format(starpath))))

    # print(specs)
    # print(mods)
    # print(seds)
    # print(xrays)
    
    
    starts = []
    ends = []

    for spec in specs:
        hdul = fits.open(spec)
        hdr = hdul[0].header
        ra, dec = hdr['RA_TARG'], hdr['DEC_TARG']
        starts.append(hdr['EXPSTART'])
        ends.append(hdr['EXPEND'])
        hdr.append(('DOI', '10.17909/T9DG6F'))
        hdr.append(('RADESYS' , 'ICRS'))
        hdr.append(('TIMESYS', 'UTC'))
        hdr.append(('HLSPVER', 'V{}'.format(version)))
        hdr.append(('HSLPID', 'MUSCLES'))
        hdr.append(('LICENSE','CC BY 4.0'))
        hdr.append(('LICENURL', 'https://creativecommons.org/licenses/by/4.0/'))
        if hdr['TARGNAME'] == '2MASS-J23062928-0502285':
            hdr['TARGNAME'] = 'TRAPPIST-1'
        new_name = (os.path.split(spec)[1]).replace('v{}'.format(oldv), 'v{}'.format(version))
        savepath = '{}{}/{}'.format(newpath, star, new_name)
        hdul.writeto(savepath, overwrite=True)
    
    for spec in mods:
        hdul = fits.open(spec)
        hdr = hdul[0].header
        hdr.append(('DOI', '10.17909/T9DG6F'))
        hdr.append(('HLSPVER', 'V{}'.format(version)))
        hdr.append(('HSLPID', 'MUSCLES'))
        hdr.append(('LICENSE','CC BY 4.0'))
        hdr.append(('LICENURL', 'https://creativecommons.org/licenses/by/4.0/'))
        if hdr['TARGNAME'] == '2MASS-J23062928-0502285':
            hdr['TARGNAME'] = 'TRAPPIST-1'
        new_name = (os.path.split(spec)[1]).replace('v{}'.format(oldv), 'v{}'.format(version))
        savepath = '{}{}/{}'.format(newpath, star, new_name)
        hdul.writeto(savepath, overwrite=True)
    
    for spec in xrays:
        hdul = fits.open(spec)
        hdr = hdul[0].header
        starts.append(hdr['EXPSTART'])
        ends.append(hdr['EXPEND'])
        hdr.append(('RA_TARG', ra))
        hdr.append(('DEC_TARG', dec))
        hdr.append(('DOI', '10.17909/T9DG6F'))
        hdr.append(('RADESYS' , 'ICRS'))
        hdr.append(('TIMESYS', 'UTC'))
        hdr.append(('HLSPVER', 'V{}'.format(version)))
        hdr.append(('HSLPID', 'MUSCLES'))
        hdr.append(('LICENSE','CC BY 4.0'))
        hdr.append(('LICENURL', 'https://creativecommons.org/licenses/by/4.0/'))
        if hdr['TARGNAME'] == '2MASS-J23062928-0502285':
            hdr['TARGNAME'] = 'TRAPPIST-1'
        new_name = (os.path.split(spec)[1]).replace('v{}'.format(oldv), 'v{}'.format(version))
        savepath = '{}{}/{}'.format(newpath, star, new_name)
        hdul.writeto(savepath, overwrite=True)
    
    start = np.min(starts)
    end = np.max(ends)
    
    for spec in seds:
        hdul = fits.open(spec)
        hdr = hdul[0].header
        hdr.append(('MJD-BEG', start))
        hdr.append(('MJD-END', end))
        hdr.append(('DOI', '10.17909/T9DG6F'))
        hdr.append(('RADESYS' , 'ICRS'))
        hdr.append(('TIMESYS', 'UTC'))
        hdr.append(('HLSPVER', 'V{}'.format(version)))
        hdr.append(('HSLPID', 'MUSCLES'))
        hdr.append(('LICENSE','CC BY 4.0'))
        hdr.append(('LICENURL', 'https://creativecommons.org/licenses/by/4.0/'))
        new_name = (os.path.split(spec)[1]).replace('v{}'.format(oldv), 'v{}'.format(version))
        for i in hdr:
            if hdr[i] in ['mod', 'Mod', 'MOD']:
                hdr[i] = 'MODEL'
        starname = hdr['TARGNAME']
        if starname == 'L-980-5':
            starname = 'L 980-5'
        elif starname == 'LHS-2686':
            starname = 'LHS 2686'
        elif starname == '2MASS-J23062928-0502285':
            starname = 'TRAPPIST-1'
        simtab = Simbad.query_object(starname)
        coords = SkyCoord(simtab['RA'][0], simtab['DEC'][0], unit=(u.hourangle, u.deg))
        if hdr['TARGNAME'] == '2MASS-J23062928-0502285':
            hdr['TARGNAME'] = 'TRAPPIST-1'

        hdr['RA_TARG'] = coords.ra.deg
        hdr['DEC_TARG'] = coords.dec.deg
        
      
        savepath = '{}{}/{}'.format(newpath, star, new_name)
        hdul.writeto(savepath, overwrite=True)
print('done')

TRAPPIST-1
done


In [7]:
# Simbad.query_objects(stars)

In [20]:
len(stars)

12

In [21]:
for i in hdr:
    if hdr[i] in ['mod', 'Mod', 'MOD']:
        print (i, hdr[i])
        hdr[i] = 'MODEL'
        print (i, hdr[i])

In [22]:
hdr

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
TELESCOP= 'MULTI   '                                                            
INSTRUME= 'MULTI   '                                                            
GRATING = 'MULTI   '                                                            
TELESC00= 'HST     '                                                            
INSTRU00= 'COS     '                                                            
GRATIN00= 'G130M   '                                                            
TELESC01= 'HST     '                                                            
INSTRU01= 'COS     '                                                            
GRATIN01= 'G160M   '        

In [11]:
c = SkyCoord(star_list['RA'][-1], star_list['DEC'][-1], unit=(u.hourangle, u.deg))
c

<SkyCoord (ICRS): (ra, dec) in deg
    (107.60025167, -39.09738083)>

In [12]:
c.dec.deg


-39.09738083333333

In [6]:
# for star in stars:
#     print(star)
#     starpath = '{}{}/'.format(inpath, star)
#     seds = glob.glob('{}*multi*.fits'.format(starpath))
#     for sed in seds:
#         hdul = fits.open(sed)
#         hdr = hdul[0].header
#         starname = hdr['TARGNAME']
#         print(starname)
#         if starname == 'L-980-5':
#             starname = 'L 980-5'
#         elif starname == 'LHS-2686':
#             starname = 'LHS 2686'
#         elif starname == '2MASS-J23062928-0502285':
#             starname = 'TRAPPIST-1'
#         simtab = Simbad.query_object(starname)
#         coords = SkyCoord(simtab['RA'][0], simtab['DEC'][0], unit=(u.hourangle, u.deg))
#         print(coords.ra.deg, coords.dec.deg)
#         print(hdr['RA_TARG'], hdr['DEC_TARG'])
#         # print(startab)

In [14]:
# simtab

In [15]:
# Simbad.query_object('TOI-134')

fix targnames in TRAPPIST-1 files:

Need to fix coords in X-ray files. XMM files don't have any values, CXO just have a DEC and no RA keyword. Have to modify the v24 individually then run the updator above without the RA and DeC input

GJ15A 4.615833 44.025056
LHS2686 197.5475 47.751917
GJ699 269.447917 4.746194
GJ849 332.42375 -4.640833
GJ163 62.324167 -53.371111


In [40]:
xrcoords = dict(GJ729=(282.45925,-23.837194),  GJ674=(262.170708,-46.899694),
GJ649 =(254.536208,25.741583),GJ1132=(153.716667,-47.156889))
xrcoords['TRAPPIST-1'] = (346.627083,-5.043861)
newpath = outpath

cxopath = '/media/david/2tb_ext_hd/hddata/mega_muscles/xray/cxo/'
datasets = os.listdir(cxopath)
for dataset in datasets:
    evt = glob.glob('{}{}/primary/*evt2.fits.gz'.format(cxopath, dataset))[0]
    hdr = fits.getheader(evt, 1)
    target = hdr['OBJECT']
    if target == '2MASS J14211512-0107199':
        target = 'L-980-5'
    if target == 'LHS2686':
        target = 'LHS-2686'
    # print(hdr['OBJECT'], hdr['RA_TARG'], hdr['DEC_TARG'])
    xrcoords[target] = (hdr['RA_TARG'], hdr['DEC_TARG'])


# TRAPPIST-1=(346.627083,-5.043861)
print(xrcoords)


for star in stars:
    if star != 'GJ676A':
        print(star)
        starpath = '{}{}/'.format(inpath, star)
        
        xrays = np.hstack((glob.glob('{}*cxo*.fits'.format(starpath)),glob.glob('{}*xmm*.fits'.format(starpath))))
        print(xrays)
        coords = xrcoords[star]
        print(coords)
        


        for spec in xrays:
            hdul = fits.open(spec)
            hdr = hdul[0].header
            
            print(hdr['RA_TARG'], hdr['DEC_TARG'])
            del hdr['RA_TARG']
            del hdr['DEC_TARG']
    #     #     starts.append(hdr['EXPSTART'])
    #     #     ends.append(hdr['EXPEND'])
            hdr.append(('RA_TARG', coords[0]))
            hdr.append(('DEC_TARG', coords[1]))
            print(hdr['RA_TARG'], hdr['DEC_TARG'])
            
            hdr.append(('DOI', '10.17909/T9DG6F'))
            hdr.append(('RADESYS' , 'ICRS'))
            hdr.append(('TIMESYS', 'UTC'))
            hdr.append(('HLSPVER', 'V{}'.format(version)))
            hdr.append(('HSLPID', 'MUSCLES'))
            hdr.append(('LICENSE','CC BY 4.0'))
            hdr.append(('LICENURL', 'https://creativecommons.org/licenses/by/4.0/'))
            if hdr['TARGNAME'] == '2MASS-J23062928-0502285':
                hdr['TARGNAME'] = 'TRAPPIST-1'
            new_name = (os.path.split(spec)[1]).replace('v{}'.format(oldv), 'v{}'.format(version))
            savepath = '{}{}/{}'.format(newpath, star, new_name)
            hdul.writeto(savepath, overwrite=True)

{'GJ729': (282.45925, -23.837194), 'GJ674': (262.170708, -46.899694), 'GJ649': (254.536208, 25.741583), 'GJ1132': (153.716667, -47.156889), 'TRAPPIST-1': (346.627083, -5.043861), 'L-980-5': (215.314167, -1.125056), 'GJ15A': (4.615833, 44.025056), 'LHS-2686': (197.5475, 47.751917), 'GJ699': (269.447917, 4.746194), 'GJ849': (332.42375, -4.640833), 'GJ163': (62.324167, -53.371111)}
GJ1132
['/media/david/2tb_ext_hd/hddata/mega_muscles/v24_hlsp/GJ1132/hlsp_muscles_xmm_epic_gj1132_na_v24_component-spec.fits']
(153.716667, -47.156889)
 
153.716667 -47.156889
GJ15A
['/media/david/2tb_ext_hd/hddata/mega_muscles/v24_hlsp/GJ15A/hlsp_muscles_cxo_acis_gj15a_none_v24_component-spec.fits']
(4.615833, 44.025056)
4.615833 44.025056
4.615833 44.025056
GJ163
['/media/david/2tb_ext_hd/hddata/mega_muscles/v24_hlsp/GJ163/hlsp_muscles_cxo_acis_gj163_none_v24_component-spec.fits']
(62.324167, -53.371111)
62.324167 -53.371111
62.324167 -53.371111
GJ649
['/media/david/2tb_ext_hd/hddata/mega_muscles/v24_hlsp/GJ6

In [46]:
for star in stars:
    if star not in ['GJ676A', 'L-980-5']:
        print('-----------------------------------------------------------------------------------------')
        
        print(star)
        print('-----------------------------------------------------------------------------------------')
        starpath = '{}{}/'.format(newpath, star)
        
        xrays = np.hstack((glob.glob('{}*cxo*.fits'.format(starpath)),glob.glob('{}*xmm*v25*.fits'.format(starpath))))
        print(xrays[0])
        hdr = fits.getheader(xrays[0])
        print(hdr)

-----------------------------------------------------------------------------------------
GJ1132
-----------------------------------------------------------------------------------------
/media/david/2tb_ext_hd/hddata/mega_muscles/v26_hlsp_fixed/GJ1132/hlsp_muscles_xmm_epic_gj1132_na_v25_component-spec.fits
SIMPLE  =                    T / conforms to FITS standard                      BITPIX  =                    8 / array data type                                NAXIS   =                    0 / number of array dimensions                     EXTEND  =                    T                                                  TELESCOP= 'XMM     '                                                            INSTRUME= 'EPIC    '                                                            GRATING = 'NA      '                                                            DETECTOR= 'MULTI   '                                                            DETECT00= 'PN      '                               